In [9]:
import pandas as pd
import numpy as np

In [3]:
def pregunta_01():
    """
    Carga de datos.
    -------------------------------------------------------------------------------------
    """

    # Lea el archivo `amazon_cells_labelled.tsv` y cree un DataFrame usando pandas.
    # Etiquete la primera columna como `msg` y la segunda como `lbl`. Esta función
    # retorna el dataframe con las dos columnas.
    df = pd.read_table(
        "amazon_cells_labelled.tsv",
        sep="\t",
        header=None,
        names=["msg", "lbl"],
    )

    # Separe los grupos de mensajes etiquetados y no etiquetados.
    df_tagged = df[df["lbl"].notnull()]
    df_untagged = df[df["lbl"].isna()]

    x_tagged = df_tagged["msg"]
    y_tagged = df_tagged["lbl"]

    x_untagged = df_untagged["msg"]
    y_untagged = df_untagged["lbl"]

    # Retorne los grupos de mensajes
    return x_tagged, y_tagged, x_untagged, y_untagged

pregunta_01()

(1       So there is no way for me to plug it in here i...
 2                             Good case, Excellent value.
 5                                  Great for the jawbone.
 10      Tied to charger for conversations lasting more...
 11                                      The mic is great.
                               ...                        
 2901    The screen does get smudged easily because it ...
 2906    What a piece of junk.. I lose more calls on th...
 2910                         Item Does Not Match Picture.
 2911    The only thing that disappoint me is the infra...
 2913    You can not answer calls with the unit, never ...
 Name: msg, Length: 1000, dtype: object,
 1       0.0
 2       1.0
 5       1.0
 10      0.0
 11      1.0
        ... 
 2901    0.0
 2906    0.0
 2910    0.0
 2911    0.0
 2913    0.0
 Name: lbl, Length: 1000, dtype: float64,
 0        I try not to adjust the volume setting to avoi...
 3              I thought Motorola made reliable products!.
 4   

In [23]:
def pregunta_02():
    """
    Preparación de los conjuntos de datos.
    -------------------------------------------------------------------------------------
    """

    # Importe train_test_split
    from sklearn.model_selection import train_test_split

    # Cargue los datos generados en la pregunta 01.
    x_tagged, y_tagged, x_untagged, y_untagged = pregunta_01()

    # Divida los datos de entrenamiento y prueba. La semilla del generador de números
    # aleatorios es 12345. Use el 10% de patrones para la muestra de prueba.
    x_train, x_test, y_train, y_test = train_test_split(
        x_tagged,
        y_tagged,
        test_size = 0.1,
        random_state=12345,
    )

    # Retorne `X_train`, `X_test`, `y_train` y `y_test`
    return x_train, x_test, y_train, y_test

pregunta_02()

(256     Buyer Beware, you could flush money right down...
 2720    I was very impressed with the price of the cases.
 1856              I am more than happy with this product.
 731     I was very excited to get this headset because...
 2713                               Buttons are too small.
                               ...                        
 1308                                   Price is good too.
 339               The BT headset was such a disapoinment.
 900                                         Great phone!.
 1474                                    A Disappointment.
 1468    After charging overnight, these batteries work...
 Name: msg, Length: 900, dtype: object,
 402     The phone takes FOREVER to charge like 2 to 5 ...
 2386    My father has the V265, and the battery is dying.
 835     Not only will it drain your player, but may al...
 1424    I find this inexcusable and so will probably b...
 2722                  Also makes it easier to hold on to.
                

In [24]:
def pregunta_03():
    """
    Construcción de un analizador de palabras
    -------------------------------------------------------------------------------------
    """
    # Importe el stemmer de Porter
    # Importe CountVectorizer

    from nltk.stem.porter import PorterStemmer
    from sklearn.feature_extraction.text import CountVectorizer

    # Cree un stemeer que use el algoritmo de Porter.
    stemmer = PorterStemmer()

    # Cree una instancia del analizador de palabras (build_analyzer)
    analyzer = CountVectorizer().build_analyzer()

    # Retorne el analizador de palabras
    return lambda x: (stemmer.stem(w) for w in analyzer(x))

pregunta_03()

<function __main__.pregunta_03.<locals>.<lambda>(x)>

In [25]:
from tkinter import Grid


def pregunta_04():
    """
    Especificación del pipeline y entrenamiento
    -------------------------------------------------------------------------------------
    """

    # Importe CountVetorizer
    # Importe GridSearchCV
    # Importe Pipeline
    # Importe BernoulliNB
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.model_selection import GridSearchCV
    from sklearn.pipeline import Pipeline
    from sklearn.naive_bayes import BernoulliNB

    # Cargue las variables.
    x_train, x_test, y_train, y_test = pregunta_02()

    # Obtenga el analizador de la pregunta 3.
    analyzer = pregunta_03()

    # Cree una instancia de CountVectorizer que use el analizador de palabras
    # de la pregunta 3. Esta instancia debe retornar una matriz binaria. El
    # límite superior para la frecuencia de palabras es del 100% y un límite
    # inferior de 5 palabras. Solo deben analizarse palabras conformadas por
    # letras.

    countVectorizer =CountVectorizer(
        analyzer=analyzer,
        lowercase=True,
        stop_words="english",
        token_pattern="(?u)\b\w\w+\b",
        binary=True,
        max_df=1.0,
        min_df=5,
    )

    # Cree un pipeline que contenga el CountVectorizer y el modelo de BernoulliNB.
    pipeline = Pipeline(
        steps=[
            ("vect", countVectorizer),
            ("clf", BernoulliNB()),
        ],
    )

    # Defina un diccionario de parámetros para el GridSearchCV. Se deben
    # considerar 10 valores entre 0.1 y 1.0 para el parámetro alpha de
    # BernoulliNB.
    param_grid = {
        "clf__alpha": np.linspace(0.1, 1.0, 10),
    }

    # Defina una instancia de GridSearchCV con el pipeline y el diccionario de
    # parámetros. Use cv = 5, y "accuracy" como métrica de evaluación
    gridSearchCV = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=5,
        scoring="accuracy",
        refit=True,
        return_train_score=False,
    )

    # Búsque la mejor combinación de regresores
    gridSearchCV.fit(x_train, y_train)

    # Retorne el mejor modelo
    return gridSearchCV

pregunta_04()

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(analyzer=<function pregunta_03.<locals>.<lambda> at 0x000002355A0CBC40>,
                                                        binary=True, min_df=5,
                                                        stop_words='english',
                                                        token_pattern='(?u)\x08\\w\\w+\x08')),
                                       ('clf', BernoulliNB())]),
             param_grid={'clf__alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
             scoring='accuracy')

In [27]:
def pregunta_05():
    """
    Evaluación del modelo
    -------------------------------------------------------------------------------------
    """

    # Importe confusion_matrix
    from sklearn.metrics import confusion_matrix

    # Obtenga el pipeline de la pregunta 3.
    gridSearchCV = pregunta_04()

    # Cargue las variables.
    x_train, x_test, y_train, y_test = pregunta_02()

    # Evalúe el pipeline con los datos de entrenamiento usando la matriz de confusion.
    cfm_train = confusion_matrix(
        y_true=y_train,
        y_pred=gridSearchCV.predict(x_train),
    )

    cfm_test = confusion_matrix(
        y_true=y_test,
        y_pred=gridSearchCV.predict(x_test),
    )

    # Retorne la matriz de confusion de entrenamiento y prueba
    return cfm_train, cfm_test

pregunta_05()

(array([[394,  60],
        [ 51, 395]], dtype=int64),
 array([[32, 14],
        [ 9, 45]], dtype=int64))

In [28]:
def pregunta_06():
    """
    Pronóstico
    -------------------------------------------------------------------------------------
    """

    # Obtenga el pipeline de la pregunta 3.
    gridSearchCV = pregunta_04()

    # Cargue los datos generados en la pregunta 01.
    x_tagged, y_tagged, X_untagged, y_untagged = pregunta_01()

    # pronostique la polaridad del sentimiento para los datos
    # no etiquetados
    y_untagged_pred = gridSearchCV.predict(X_untagged)

    # Retorne el vector de predicciones
    return y_untagged_pred

pregunta_06()

array([0., 1., 1., ..., 1., 0., 0.])